<a href="https://colab.research.google.com/github/mrarat76/Tensorflow_Practises/blob/main/NLP_Practise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
## recurrent neural network

## LSTM rnn hücrelerine denir "tanh" non-linearity eklemek için kullnaırız


## Outputta sigmodi kullanıyoruz


NLP'nin amacı bilgiyi doğal dilden elde etmektir( metini veya konuşmayı sıralamak gibi)

NLP genel manada seq2seq problemlerine odaklanır.


**Seq2seq**, kısaltması "sequence to sequence" (diziden diziye) anlamına gelir ve genellikle doğal dil işleme (NLP) ve makine öğrenmesinde kullanılan bir model türüdür. Bu model, bir girdi dizisini (örneğin bir cümle veya kelime dizisi) alır ve başka bir diziye (genellikle başka bir dilde bir cümle veya metin) çevirir. Seq2seq modelleri, özellikle makine çevirisi, otomatik özetleme, soru-cevap sistemleri ve sohbet robotları gibi alanlarda yaygın olarak kullanılır.

Seq2seq modeli tipik olarak iki ana bileşenden oluşur:

Encoder (Kodlayıcı): Girdi dizisini alır ve bu diziyi sabit boyutlu bir vektör temsiline dönüştürür. Bu vektör, girdi dizisinin bir tür özeti olarak düşünülebilir ve genellikle diziye ait önemli bilgileri içerir.

Decoder (Çözücü): Encoder tarafından oluşturulan sabit boyutlu vektörü alır ve hedef diziye dönüştürür. Çözücü, bu vektörü kullanarak adım adım çıktı dizisini üretir.

Bu modeller genellikle tekrarlayan sinir ağları (RNN), uzun kısa süreli bellek ağları (LSTM) veya gated recurrent units (GRU) gibi yapıları kullanır. Son yıllarda, daha gelişmiş seq2seq modelleri, dikkat mekanizmaları (attention mechanisms) ve Transformer mimarileri gibi teknikleri kullanarak daha iyi performans sergilemiştir. Dikkat mekanizması, modelin girdi dizisinin hangi bölümlerine odaklanması gerektiğini daha iyi anlamasını sağlar ve böylece daha doğru çeviriler yapılmasına olanak tanır.







In [3]:
!nvidia-smi -L

/bin/bash: line 1: nvidia-smi: command not found


In [4]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py


--2024-03-09 17:05:52--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-03-09 17:05:53 (98.2 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [5]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [6]:
# Download data (same as from Kaggle)
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzip data
unzip_data("nlp_getting_started.zip")

--2024-03-09 17:05:57--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 209.85.200.207, 209.85.234.207, 108.177.112.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|209.85.200.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2024-03-09 17:05:57 (90.3 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [7]:
### Visualizing a text dataset

import pandas as pd

train_df=pd.read_csv("train.csv")
test_df=pd.read_csv("test.csv")

train_df.head()



,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [8]:
train_df["text"][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [9]:
## Dataframi shuffellama

train_df_shuffled =train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()



,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [10]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [11]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [12]:
## How many total samples?


len(train_df), len(test_df)

(7613, 3263)

In [13]:
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target =row
  print(f"Target:{target}", "(real_diaster)" if target > 0 else "(not real diaster)")
  print(f"Text\n{text}\n")
  print("---\n")



Target:0 (not real diaster)
Text
Tried orange aftershock today. My life will never be the same

---

Target:0 (not real diaster)
Text
wowo--=== 12000 Nigerian refugees repatriated from Cameroon

---

Target:0 (not real diaster)
Text
Season 3 of New Girl was such a emotional train wreck I just wanted to cry laugh and eat a lot of ice cream

---

Target:1 (real_diaster)
Text
Jack Wilshere has poor injury recordand his off field behaviors doesn't help.#Arsenal

---

Target:1 (real_diaster)
Text
#Flood in Bago Myanmar #We arrived Bago

---



In [14]:
from sklearn.model_selection import train_test_split

# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # dedicate 10% of samples to validation set
                                                                            random_state=42)

In [15]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [16]:
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

In [17]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization # after TensorFlow 2.6

# Before TensorFlow 2.6
# from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
# Note: in TensorFlow 2.6+, you no longer need "layers.experimental.preprocessing"
# you can use: "tf.keras.layers.TextVectorization", see https://github.com/tensorflow/tensorflow/releases/tag/v2.6.0 for more

# Use the default TextVectorization variables
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (all of the different words in your text)
                                    standardize="lower_and_strip_punctuation", # how to process text
                                    split="whitespace", # how to split tokens
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None) # how long should the output sequence of tokens be?
                                    # pad_to_max_tokens=True) # Not valid if using max_tokens=None

In [18]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))


15

In [19]:
max_vocab_lenght= 10000
max_length=15

text_vectorizer= TextVectorization(max_tokens= max_vocab_lenght,
                                   output_mode="int",
                                   output_sequence_length= max_length)


In [20]:
## Fit the text vectorizer

text_vectorizer.adapt(train_sentences)


sample_sentence="There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [21]:
# Random sentence choose


random_sentence = random.choice(train_sentences)
print(f"Original text: \n {random_sentence}\
\n\nVectorized version:")

text_vectorizer([random_sentence])

Original text: 
 Broadcast journalism: hostages to fortune otherwise quot-television blind else quot-operations since-3g superv...

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   1,    1,  422,    5, 5610, 4979, 9409, 2539, 1026, 9410, 8495,
        7915,    0,    0,    0]])>

In [22]:
## Get the unique words in the vocab


words_in_vocab =text_vectorizer.get_vocabulary()

top_5_words = words_in_vocab[5:]
bottom_5_words = words_in_vocab[-5:]
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"5 most common words: {top_5_words}")
print(f"5 least common words:{bottom_5_words}")

Number of words in vocab: 10000
5 most common words: ['to', 'of', 'and', 'i', 'is', 'for', 'on', 'you', 'my', 'with', 'it', 'that', 'at', 'by', 'this', 'from', 'be', 'are', 'was', 'have', 'like', 'as', 'up', 'so', 'just', 'but', 'me', 'im', 'your', 'not', 'amp', 'out', 'its', 'will', 'an', 'no', 'has', 'fire', 'after', 'all', 'when', 'we', 'if', 'now', 'via', 'new', 'more', 'get', 'or', 'about', 'what', 'he', 'people', 'news', 'been', 'over', 'one', 'how', 'dont', 'they', 'who', 'into', 'were', 'do', 'us', '2', 'can', 'video', 'emergency', 'there', 'disaster', 'than', 'police', 'would', 'his', 'still', 'her', 'some', 'body', 'storm', 'crash', 'burning', 'suicide', 'back', 'man', 'california', 'why', 'time', 'them', 'had', 'buildings', 'rt', 'first', 'cant', 'see', 'got', 'day', 'off', 'our', 'going', 'nuclear', 'know', 'world', 'bomb', 'fires', 'love', 'killed', 'go', 'attack', 'youtube', 'dead', 'two', 'families', '3', 'train', 'full', 'being', 'war', 'many', 'today', 'think', 'only',

In [23]:
train_sentences

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       ...,
       'Near them on the sand half sunk a shattered visage lies... http://t.co/0kCCG1BT06',
       "kesabaran membuahkan hasil indah pada saat tepat! life isn't about waiting for the storm to pass it's about learning to dance in the rain.",
       "@ScottDPierce @billharris_tv @HarrisGle @Beezersun I'm forfeiting this years fantasy football pool out of fear I may win n get my ass kicked"],
      dtype=object)

Yukarıda yaptıklarımız tokenization layer oluyor.


In [24]:
### Embedding layer
tf.random.set_seed(42)
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=max_length, # how long is each input
                             name="embedding_1")

embedding


In [26]:
# Get a random sentence from training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
Officer Wounded Suspect Killed in Exchange of Gunfire: Richmond police officer wounded suspect killed in exc... http://t.co/zDHwRN6cZc      

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 3.6254417e-02, -1.9992245e-02, -1.8053092e-02, ...,
         -5.4149516e-03, -2.5704814e-02, -4.2600539e-02],
        [-4.2949118e-02, -4.8324835e-02,  4.2365339e-02, ...,
         -1.0115780e-02,  4.7386888e-02, -3.1728528e-02],
        [-2.9953886e-02,  2.4106029e-02,  1.8173829e-03, ...,
         -2.2880722e-02, -1.8537045e-05, -4.2054463e-02],
        ...,
        [-2.9953886e-02,  2.4106029e-02,  1.8173829e-03, ...,
         -2.2880722e-02, -1.8537045e-05, -4.2054463e-02],
        [-1.5992571e-02, -2.1838618e-02, -3.3072591e-02, ...,
         -1.2810908e-02,  1.9084159e-02,  8.4461682e-03],
        [ 8.1163049e-03, -1.6408108e-02,  2.2676017e-02, ...,
         -3.0414987e-02, -2.6391555e-02,  3.6384333e-02]]], dtype=float32)>

In [27]:
sample_embed[0][0]


<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([ 0.03625442, -0.01999225, -0.01805309,  0.02101442, -0.01019566,
       -0.01675968, -0.00208356, -0.00723351,  0.02718829,  0.00109817,
       -0.01518078,  0.02224455,  0.02202222,  0.04090952,  0.01499469,
       -0.00713081,  0.0374749 ,  0.01039122, -0.00072841,  0.00545859,
       -0.02518255,  0.00355673, -0.01210935,  0.02372396,  0.01150034,
       -0.04544449, -0.0430079 ,  0.0224801 , -0.01890893,  0.01880899,
       -0.03841449,  0.0142321 , -0.01701351, -0.03244992,  0.02479258,
        0.04766661,  0.04261366,  0.04545504,  0.03876133,  0.00137961,
        0.02133824,  0.00457326, -0.02148392, -0.02712168, -0.00908245,
       -0.0115482 , -0.03806983, -0.04135561,  0.00901185,  0.00599109,
        0.00420856, -0.02473085,  0.02637054,  0.03422799, -0.00714142,
       -0.0343008 ,  0.0167567 ,  0.02458973, -0.0364904 ,  0.03396695,
       -0.04904422, -0.02940992,  0.01470626,  0.02483178,  0.02423284,
       -0.003644

In [28]:
## Text verisini modelleme (running a series of experiments)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
                    ("clf", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)


Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [29]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [30]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [31]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  -----
  y_true = true labels in the form of a 1D array
  y_pred = predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [32]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}